In [4]:

import json
import csv
import xml.etree.ElementTree as ET
from pony.orm import *
import hashlib
import re 
 
db = Database(
    provider="mysql",
    host="127.0.0.1",
    user="root",
    password="Kangroo13",
    database="basecar",
)
 
class Records2(db.Entity):
    first_name = Required(str)
    last_name = Required(str)
    age = Optional(int)
    sex = Optional(str)
    vehicle_make = Optional(str)
    vehicle_model = Optional(str)
    vehicle_year = Optional(int)
    vehicle_type = Optional(str)
    iban = Optional(str)
    credit_card_number = Optional(str)
    credit_card_security_code = Optional(str)
    credit_card_start_date = Optional(str)
    credit_card_end_date = Optional(str)
    address_main = Optional(str)
    address_city = Optional(str)
    address_postcode = Optional(str)
    debt_amount = Optional(float)
    time_period_years = Optional(int)
    retired = Optional(bool)
    dependants = Optional(int)
    marital_status = Optional(str)
    salary = Optional(float)
    pension = Optional(float)
    company = Optional(str)
    commute_distance = Optional(float)
 
db.generate_mapping(create_tables=True)

def sanitize_input(input_str):#change function names
    return re.sub(r'[^a-zA-Z0-9]', '', input_str)

def hash_credit_card_number(credit_card_number):#change function names
    sha256 = hashlib.sha256()
    sha256.update(credit_card_number.encode('utf-8')) 
    return sha256.hexdigest()





 
@db_session
def read_csv_and_insert_into_db(csv_filename):
    with open(csv_filename, 'r', newline='') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            existing_data = Records2.get(first_name=row['First Name'], last_name=row['Second Name'])
            if existing_data:
                existing_data.set(
                    age=int(row['Age (Years)']),
                    sex=row['Sex'],
                    vehicle_make=row['Vehicle Make'],
                    vehicle_model=row['Vehicle Model'],
                    vehicle_year=int(row['Vehicle Year']),
                    vehicle_type=row['Vehicle Type']
                )
            else:
                Records2(
                    first_name=row['First Name'],
                    last_name=row['Second Name'],
                    age=int(row['Age (Years)']),
                    sex=row['Sex'],
                    vehicle_make=row['Vehicle Make'],
                    vehicle_model=row['Vehicle Model'],
                    vehicle_year=int(row['Vehicle Year']),
                    vehicle_type=row['Vehicle Type']
                )
 
read_csv_and_insert_into_db('user_data.csv')
 
@db_session
def read_json_and_insert_into_db(json_filename):
    with open(json_filename, 'r') as json_file:
        data = json.load(json_file)
        for item in data:
            existing_data = Records2.get(first_name=item["firstName"], last_name=item["lastName"])
            if existing_data:
                existing_data.set(
                    age=item["age"],
                    iban=sanitize_input(item["iban"]),
                    credit_card_number=hash_credit_card_number(item["credit_card_number"]),
                    credit_card_security_code=item["credit_card_security_code"],
                    credit_card_start_date=item["credit_card_start_date"],
                    credit_card_end_date=item["credit_card_end_date"],
                    address_main=item["address_main"],
                    address_city=item["address_city"],
                    address_postcode=item["address_postcode"]
                )
            else:
                Records2(
                    first_name=item["firstName"],
                    last_name=item["lastName"],
                    age=item["age"],
                    iban=sanitize_input(item["iban"]),# must do same for rest of them 
                    credit_card_number=hash_credit_card_number(item["credit_card_number"]),
                    credit_card_security_code=item["credit_card_security_code"],
                    credit_card_start_date=item["credit_card_start_date"],
                    credit_card_end_date=item["credit_card_end_date"],
                    address_main=item["address_main"],
                    address_city=item["address_city"],
                    address_postcode=item["address_postcode"]
                )
 
read_json_and_insert_into_db('user_data.json')
 
@db_session
def read_xml_and_insert_into_db(xml_filename):
    tree = ET.parse(xml_filename)
    root = tree.getroot()
    for user in root.findall('user'):
        first_name = user.get('firstName')
        last_name = user.get('lastName')
        existing_data = Records2.get(first_name=first_name, last_name=last_name)
        if existing_data:
            existing_data.set(
                age=int(user.get('age')) if user.get('age') else None,
                sex=user.get('sex'),
                retired=user.get('retired') == 'True' if user.get('retired') else None,
                dependants=int(user.get('dependants')) if user.get('dependants') else None,
                marital_status=user.get('marital_status'),
                salary=int(user.get('salary')) if user.get('salary') else None,
                pension=int(user.get('pension')) if user.get('pension') else None,
                company=user.get('company'),
                commute_distance=float(user.get('commute_distance')) if user.get('commute_distance') else None,
                address_postcode=user.get('address_postcode')
            )
        else:
            Records2(
                first_name=first_name,
                last_name=last_name,
                age=int(user.get('age')) if user.get('age') else None,
                sex=user.get('sex'),
                retired=user.get('retired') == 'True' if user.get('retired') else None,
                dependants=int(user.get('dependants')) if user.get('dependants') else None,
                marital_status=user.get('marital_status'),
                salary=int(user.get('salary')) if user.get('salary') else None,
                pension=int(user.get('pension')) if user.get('pension') else None,
                company=user.get('company'),
                commute_distance=float(user.get('commute_distance')) if user.get('commute_distance') else None,
                address_postcode=user.get('address_postcode')
            )
 
read_xml_and_insert_into_db('user_data.xml')